# Intro to gragh convolutional surrogate models
Eamon Whalen

In [1]:
import sys
import os
import numpy as np
import pandas as pd
import altair as alt

from gcnSurrogate.models.feastnetSurrogateModel import FeaStNet
from gcnSurrogate.readers.loadConmechGraphs import loadConmechGraphs
from gcnSurrogate.visualization.altTrussViz import plotTruss, interactiveErrorPlot
from gcnSurrogate.util.gcnSurrogateUtil import *

## 1. Load simulation data

In [2]:
# dataDir = 'data/2D_Truss_v1.3/conmech/design_7_N_1000/'
dataDir = 'data/endLoadsv1.0/conmech/design_7_N_1000/'
allGraphsUnfiltered = loadConmechGraphs(dataDir)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphsUnfiltered]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,1000.000000
mean,0.476726
std,4.870080
min,0.013036
25%,0.045080
50%,0.089405
75%,0.187214
max,102.305313


In [4]:
plotTruss(allGraphsUnfiltered[0], showDeformed=True, defScale=10)

alt.LayerChart(...)

## 2. Filter and partition

In [5]:
allGraphs = filterbyDisp(allGraphsUnfiltered, 0.9)
trainData, valData, testData = partitionGraphList(allGraphs)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphs]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,900.000000
mean,0.107378
std,0.088163
min,0.013036
25%,0.042086
50%,0.079557
75%,0.139971
max,0.411479


## 3. Train a GCN

In [6]:
gcn = FeaStNet()
history = gcn.trainModel(trainData, valData, 
                         epochs=100,
                         saveDir='results/gcn04-endload/')

plotHistory(history)

epoch: 0   trainLoss: 8.0628e-01   valLoss:1.1546e+00  time: 6.17e+00
epoch: 1   trainLoss: 6.1361e-01   valLoss:1.1045e+00  time: 1.63e+00
epoch: 2   trainLoss: 4.1935e-01   valLoss:1.5562e+00  time: 1.65e+00
epoch: 3   trainLoss: 3.4435e-01   valLoss:2.8491e+00  time: 1.66e+00
epoch: 4   trainLoss: 2.7464e-01   valLoss:2.8004e+00  time: 1.64e+00
epoch: 5   trainLoss: 2.4852e-01   valLoss:3.9219e+00  time: 1.67e+00
epoch: 6   trainLoss: 1.9334e-01   valLoss:5.4532e+00  time: 1.46e+00
epoch: 7   trainLoss: 1.5566e-01   valLoss:2.3938e+00  time: 1.63e+00
epoch: 8   trainLoss: 1.3486e-01   valLoss:8.5127e-01  time: 1.53e+00
epoch: 9   trainLoss: 1.1568e-01   valLoss:4.6650e-01  time: 1.65e+00
epoch: 10   trainLoss: 1.0078e-01   valLoss:4.6212e-01  time: 1.64e+00
epoch: 11   trainLoss: 9.0617e-02   valLoss:4.2002e-01  time: 1.65e+00
epoch: 12   trainLoss: 9.0460e-02   valLoss:4.8459e-01  time: 1.66e+00
epoch: 13   trainLoss: 8.4915e-02   valLoss:5.0642e-01  time: 1.63e+00
epoch: 14   trai

alt.Chart(...)

## 4. Test the GCN

In [7]:
trainRes = gcn.testModel(trainData)
testRes = gcn.testModel(testData)
pd.DataFrame([trainRes, testRes], index=['train', 'test'])

,mse,mae,mre,peakR2,maxAggR2,meanAggR2,minAggR2
train,0.000073,0.005366,0.120315,0.951026,0.985448,0.854234,0.0
test,0.000119,0.006057,0.132866,0.883616,0.969295,0.830178,0.0


## 5. Visualize some predictions

In [14]:
i = 4
pred = gcn.predict([testData[i]])[0]
plotTruss(testData[i], showDeformed=True, defScale=20, prediction=pred)

alt.LayerChart(...)

In [15]:
# interactive scatter plot
alt.data_transformers.enable('json')
allPreds = gcn.predict(testData)
display(interactiveErrorPlot(testData, allPreds))
alt.data_transformers.enable('default');

alt.HConcatChart(...)

In [17]:
trainData[0].x

tensor([[  0.,   0., -10.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        [  0.,   0., -10.],
        [  0.,   0.,   0.],
        [  1.,   1.,   0.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        [  0.,   1.,   0.],
        [  0.,   0.,   0.]], dtype=torch.float64)